Imports

In [1]:
import ray
import os
os.environ["OPENCV_LOG_LEVEL"] = "SILENT"
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
for devices in physical_devices:
    tf.config.experimental.set_memory_growth(devices, True)

from sklearn.metrics import classification_report
import tflite_runtime.interpreter as tflite
from dvclive import Live
import onnxruntime as ort

import numpy as np
import keras
import cv2

2024-02-23 08:12:44.170512: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 08:12:44.170547: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 08:12:44.170565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading dataset

In [2]:
dim =64
image_size = (dim, dim)
batch_size = 150

train_ds, other = keras.utils.image_dataset_from_directory(
    "dataset/dataset_checkin",
    validation_split=0.26,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

# train_ds = train_ds.map(lambda x, y: (tf.cast(x, tf.uint8), y))


bch = other.cardinality().numpy()
bch /= 2
test_ds, val_ds = other.take(bch), other.skip(bch)
# test_ds = test_ds.map(lambda x, y: (tf.cast(x, tf.uint8), y))

y_test = np.concatenate([y.numpy() for _, y in test_ds])
label = {0:'left', 1:'lower', 2:'right', 3:'smile', 4:'upper'}
y_test = [label[i] for i in y_test]

X_test = np.concatenate([x.numpy() for x, _ in test_ds])
X_test = np.expand_dims(X_test, axis=1)

Found 27252 files belonging to 5 classes.
Using 20167 files for training.
Using 7085 files for validation.


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


Video

In [ ]:
Images = list()
cap = cv2.VideoCapture('Video/one.MOV')
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break
    Images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

Images_resized = np.array(list(map(lambda x: cv2.resize(x, (768, 768)), Images)))


# TFlite
Model Conversion

In [3]:
def representative_data_gen():
  for input_value, _ in train_ds.take(10):
    yield [input_value]

keras_model = keras.models.load_model('Models/64Efficientnetv2B0.keras')
print('loaded')
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)  # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
print('Data loaded')
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input utput tensors to uint8
converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.int16
print('converting')
tflite_model = converter.convert()
# Save the model.
with open('Models/64Efficientnetv2B0_quint8.tflite', 'wb') as f:
    f.write(tflite_model)

loaded
Data loaded
converting
INFO:tensorflow:Assets written to: /tmp/tmp_p8mharb/assets


INFO:tensorflow:Assets written to: /tmp/tmp_p8mharb/assets
/my-byte/Joel/faceid_venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: FLOAT32


Ray Inference

In [4]:
ray.init()

@ray.remote
def inference(Image):
    model = tflite.Interpreter(model_path='Models/64_8L524K_qAct16W8.tflite')

    model.allocate_tensors()
    input_details = model.get_input_details()

    model.set_tensor(input_details[0]['index'], Image)

    # Run inference
    model.invoke()

    # Get the output
    output_details = model.get_output_details()
    pred = model.get_tensor(output_details[0]['index'])
    return pred[0]

futures = [inference.remote(i) for i in X_test]
# Retrieve results.
pred_arr = ray.get(futures)
ray.shutdown()
y_pred = np.argmax(pred_arr, axis=1)
y_pred = [label[i] for i in y_pred]

(inference pid=18008) INFO: Created TensorFlow Lite XNNPACK delegate for CPU. [repeated 23x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


Without Ray Inference

In [ ]:
model = tflite.Interpreter(model_path='Model/64_8L524K_quint8.tflite', num_threads=10)
batch = 30
def inference(Image):
    model.resize_tensor_input(0, [batch, 768, 768, 3])
    model.allocate_tensors()
    input_details = model.get_input_details()

    model.set_tensor(input_details[0]['index'], Image)

    # Run inference
    model.invoke()

    # Get the output
    output_details = model.get_output_details()
    pred = model.get_tensor(output_details[0]['index'])
    return pred

prediction = inference(X_test[:batch])

DVC

In [6]:
Folder = '64_8L524k'
model_name = '64_8L524K_qAct16W8'
model_path = 'Models/'
with Live(dir=Folder+'/dvclive/', dvcyaml=Folder + '/dvc.yaml',
          exp_name='64_8L524k', resume=True) as live:
    live.log_artifact('Models/64_8L524K_qAct16W8.tflite', type='model', name=model_name)
    live.log_sklearn_plot("confusion_matrix", y_test, y_pred, name='64_8L524K_qAct16W8_tflite', title='64_8L524K_qAct16W8_tflite')
    # live.end()


Remove it from outputs to make DVCLive work as expected.


In [5]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm, cm/np.sum(cm, axis=0)*100, sep='\n')

              precision    recall  f1-score   support

        left       0.93      0.94      0.93       505
       lower       0.91      0.92      0.91       859
       right       0.96      0.93      0.95       506
       smile       0.89      0.92      0.91       772
       upper       0.94      0.92      0.93       958

    accuracy                           0.92      3600
   macro avg       0.93      0.92      0.92      3600
weighted avg       0.92      0.92      0.92      3600

[[474   7   4  16   4]
 [  5 786   8  23  37]
 [  5   9 471  16   5]
 [ 22  21   4 711  14]
 [  6  44   2  29 877]]
[[92.578125    0.80738178  0.81799591  2.01257862  0.42689434]
 [ 0.9765625  90.65743945  1.63599182  2.89308176  3.94877268]
 [ 0.9765625   1.03806228 96.3190184   2.01257862  0.53361793]
 [ 4.296875    2.42214533  0.81799591 89.43396226  1.4941302 ]
 [ 1.171875    5.07497116  0.40899796  3.64779874 93.59658485]]


# ONNX

Conversion

In [9]:
!python -m tf2onnx.convert --tflite Models/Fitment_Prediction/best_saved_model/best_integer_quant.tflite --output Models/Fitment_Prediction/best_integer_quant.onnx --opset 13

/opt/conda/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-02-08 08:34:34,279 - INFO - Using tensorflow=2.13.1, onnx=1.15.0, tf2onnx=1.16.1/15c810
2024-02-08 08:34:34,279 - INFO - Using opset <onnx, 13>
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-02-08 08:34:35,706 - INFO - Optimizing ONNX model
2024-02-08 08:34:55,741 - INFO - After optimization: Cast -9 (9->0), Const -1709 (2518->809), DequantizeLinear -64 (664->600), Identity -2 (2->0), QuantizeLinear -56 (427->371), Transpose -483 (486->3)
2024-02-08 08:34:55,894 - INFO - 
2024-02-08 08:34:55,894 - INFO - Successfully converted TensorFlow model Models/Fitment_Prediction/best_saved_model/best_integer_quant.tflite to ONNX
2024-02-08 08:34:55,894 - INFO - Model inputs: ['serving_default_images:0']
2024-02-08 08:34:55,894 - INFO -

Ray Inference

In [ ]:
ray.shutdown()
ray.init()

@ray.remote
def inference(Image):
    sess = ort.InferenceSession("Models/768Efficientnetv2B0_quint8.onnx", providers=["CUDAExecutionProvider"])
    return sess.run(["StatefulPartitionedCall:0"], {"serving_default_input_1:0": Image})[0][0]

futures = [inference.remote(i) for i in X_test]
# Retrieve results.
pred_arr = ray.get(futures)
ray.shutdown()
y_pred = np.argmax(pred_arr, axis=1)
y_pred = [label[i] for i in y_pred]

Without Ray Inference

In [10]:



sess = ort.InferenceSession("Models/Fitment_Prediction/best_int8.onnx", providers=["CUDAExecutionProvider"])
results_ort = sess.run(["PartitionedCall:0", "PartitionedCall:1"], {"serving_default_images:0": np.expand_dims(X_test[0], axis=0)})


DVC

In [ ]:
model_name = str(dim) + 'Efficientnetv2B0_quint8'
model_path = 'Models/'
with Live(exp_name=model_name, resume=True) as live:
    live.log_artifact('Models/768Efficientnetv2B0_quint8.onnx', type='model', name=model_name)
    live.log_sklearn_plot("confusion_matrix", y_test, y_pred, name='quant_onnx', title='quant_onnx')


# CORE ML

In [ ]:
import coremltools
# import onnx_coreml
coreml_model = coremltools.converters.onnx.convert("Models/768Efficientnetv2B0_quint8.onnx",
                                                    mode='classifier',
                                                    class_labels=['Left', 'Lower', 'Right', 'Smile', 'Upper']
                                                    )
# coreml_model.spec.description.output[0].type.multiArrayType.dataType = coremltools.proto.FeatureTypes_pb2.ArrayFeatureType.BOOL

coreml_model.save("Models/768Efficientnetv2B0_quint8.mlmodel")


# Fitment Prediction

## Pytorch to onnx

In [1]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('Models/Fitment Prediction/best.pt')

# Export the model to ONNX format
model.export(format='tflite', int8=True)  # creates 'yolov8n.onnx'


/newvolume/mybyte/Joel/venv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


Ultralytics YOLOv8.1.10 🚀 Python-3.10.9 torch-2.1.2+cu121 CPU (AMD EPYC 7R32)
YOLOv8x-seg summary (fused): 295 layers, 71724508 parameters, 0 gradients, 343.7 GFLOPs

PyTorch: starting from 'Models/Fitment Prediction/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 40, 8400), (1, 32, 160, 160)) (137.3 MB)

TensorFlow SavedModel: starting export with tensorflow 2.13.1...

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 19.6s, saved as 'Models/Fitment Prediction/best.onnx' (273.9 MB)
TensorFlow SavedModel: running 'onnx2tf -i "Models/Fitment Prediction/best.onnx" -o "Models/Fitment Prediction/best_saved_model" -nuo --verbosity info -oiqt -qt per-tensor'

Automatic generation of each OP name started ========================================
Automatic generation of each OP name complete!

Model loaded ========================================================================

Model conversion started ==

fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


INT8 Quantization tflite output complete!


fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


Full INT8 Quantization tflite output complete!
INT8 Quantization with int16 activations tflite output complete!
Full INT8 Quantization with int16 activations tflite output complete!


2024-02-08 07:28:22.460160: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-08 07:28:22.460260: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.129.3 does not match DSO version 535.154.5 -- cannot find working devices in this configuration


TensorFlow SavedModel: export success ✅ 959.9s, saved as 'Models/Fitment Prediction/best_saved_model' (891.9 MB)

TensorFlow Lite: starting export with tensorflow 2.13.1...
TensorFlow Lite: export success ✅ 0.0s, saved as 'Models/Fitment Prediction/best_saved_model/best_int8.tflite' (68.8 MB)

Export complete (965.6s)
Results saved to /newvolume/mybyte/Joel/MyByte-Face_ID_Screening/Models/Fitment Prediction
Predict:         yolo predict task=segment model=Models/Fitment Prediction/best_saved_model/best_int8.tflite imgsz=640 int8 
Validate:        yolo val task=segment model=Models/Fitment Prediction/best_saved_model/best_int8.tflite imgsz=640 data=/newvolume/mybyte/subbiah/MyByte-ML_Model_Training-2/data.yaml int8 
Visualize:       https://netron.app


'Models/Fitment Prediction/best_saved_model/best_int8.tflite'

In [ ]:
!pip install ultralytics

In [1]:
40*8400

336000

In [2]:
32*160*160

819200